In [1]:
neg_dir = "/home/vedic/Downloads/review_polarity/txt_sentoken/neg"
pos_dir = "/home/vedic/Downloads/review_polarity/txt_sentoken/pos"

In [2]:
import nltk
import os
from nltk.stem.porter import PorterStemmer
from collections import Counter

In [3]:
neg_rev = []
pos_rev = []

In [4]:
def read_file(file):
    text = open(file, 'r')
    text = text.read()
    return text

In [5]:
for file in os.listdir(neg_dir):
    neg_rev.append(read_file(neg_dir + '/' + file))
    
for file in os.listdir(pos_dir):
    pos_rev.append(read_file(pos_dir + '/' + file))

In [6]:
import numpy as np
neg_rev = np.asarray(neg_rev)
pos_rev = np.asarray(pos_rev)

In [7]:
import string

In [8]:
def clean_text(document):
    tokens = document.split()
    tokens = [word.lower() for word in tokens]
    tokens = [word for word in tokens if word not in string.punctuation]
    tokens = [word for word in tokens if word.isalpha()]
    stop = set(nltk.corpus.stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop]
    tokens = [word for word in tokens if len(word) > 1]
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]
    #vocab.update(tokens)
    return tokens

In [9]:
neg_rev_cleaned = []
pos_rev_cleaned = []

#vocab = Counter()

for i in range(len(neg_rev)):
    neg_rev_cleaned.append(clean_text(neg_rev[i]))
    
for i in range(len(pos_rev)):
    pos_rev_cleaned.append(clean_text(pos_rev[i]))

In [10]:
import math

In [11]:
def TfIdf_Score(word, document, review_list):
    
    Tf = document.count(word)/len(document)
        
    apps = 0
    for review in review_list:
        if word in review:
            apps += 1

    Idf = math.log(len(review_list)/apps, 10)
    
    return Tf*Idf

In [12]:
from tqdm import tqdm

In [13]:
for i in tqdm(range(len(neg_rev_cleaned))):
    neg_rev_cleaned[i] = [word for word in neg_rev_cleaned[i] if (TfIdf_Score(word, neg_rev_cleaned[i], neg_rev_cleaned) >= 0.01)]

100%|██████████| 1000/1000 [15:49<00:00,  1.50it/s]


In [14]:
for i in tqdm(range(len(pos_rev_cleaned))):
    pos_rev_cleaned[i] = [word for word in pos_rev_cleaned[i] if (TfIdf_Score(word, pos_rev_cleaned[i], pos_rev_cleaned) >= 0.01)]

100%|██████████| 1000/1000 [19:44<00:00,  1.40it/s]


In [17]:
vocab = Counter()

for rev in neg_rev_cleaned + pos_rev_cleaned:
    vocab.update(rev)

In [23]:
negs = []
poss = []
for i in range(len(neg_rev)):
    negs.append('neg')
    
for i in range(len(pos_rev)):
    poss.append('pos')
    
negs = np.asarray(negs)
poss = np.asarray(poss)

In [24]:
neg_rev = np.column_stack((neg_rev, neg_rev_cleaned, negs))
pos_rev = np.column_stack((pos_rev, pos_rev_cleaned, poss))

In [25]:
reviews = np.append(neg_rev, pos_rev, axis=0)

In [26]:
np.random.shuffle(reviews)

In [27]:
import pickle

pickle.dump(reviews, open('clean_reviews.pickle', 'wb'))
pickle.dump(vocab, open('vocabulary.pickle', 'wb'))

In [28]:
file = open('clean_reviews.pickle', "rb")
review = pickle.load(file)